## 1. Simulation-Based Inference SIR Model: Data Generation
Created: May 23, 2022 Prosper and Prosper<br>
Updated: Oct 30, 2023 HBP

### Introduction

The goal of this project is to use simulation-based inference (a.k.a., likelihood-free inference) to make reliable inferences about the SIR model when applied to the classic English Boarding School data. 

### SIR Model

This is perhaps the simplest model of an epidemic with 3 compartments: susceptible ($S$), infected ($I$), and removed ($R$).

#### Variables

\begin{align*}
    S(t) &= \mbox{mean number of susceptible persons at time $t$,}\\
    I(t) &= \mbox{mean number of infected persons at time $t$,}\\
    R(t) &= \mbox{mean number of persons removed from the infected class at time $t$,}\\
\end{align*}

#### Parameters

\begin{align*}
    \alpha &= \mbox{removal rate (due to recovery or mortality); so $1/\alpha$ is the mean infectious period,}\\
    \beta &= \mbox{transmission rate per infected person.}\\
\end{align*}

The mean number of new infections per day at time $t$ is
$\beta S(t) I(t)$.

#### Equations

\begin{align}
    \frac{dS}{dt} & = - \beta S I,\\
    \frac{dI}{dt} & = - \alpha I + \beta S I ,\\
    \frac{dR}{dt} & = \alpha I.
\end{align}

### Statistical inference
Epidemics are stochastic processes. Therefore, in principle, they can be modeled with a statistical model. Unfortunately, the statistical models are generally intractable and approximations are needed. In Ref.[1], it is shown how (frequentist) inferences can be performed with proven guarantees, namely **exact coverage** for **confidence sets** provided that one has access to an accurate simulator of the stochastic process. Crucially, knowledge of the underlying statistical model, and hence likelihood function, is not needed. 

In the context of the SIR model, one uses a continuous time Markov chain (CTMC) simulator $F_\theta$ to generate an ensemble of synthetic epidemics in which each epidemic, $j$, is associated with a randomly chosen point $\theta_j = (\alpha_j, \beta_j)$ in the parameter space $\Theta$ of the SIR model. 
Given a test statistic $\lambda({\cal D}_j;  \theta_j)$ of our choosing, where ${\cal D}_i$ denotes a time-ordered sequence of *simulated* counts of infected school children, one uses the synthetic data to estimate a variety of quantities, such as

$$P(\theta) \equiv \mathbb{P}[\lambda({\cal D}_j; \theta_j) \le \lambda(D; \theta_j)],$$ 

where $D$ denotes the sequence of *observed* counts of infected children. 


### Test statistic $\lambda({\cal D}; \theta)$

For the SIR model, with parameters $\theta \equiv \alpha, \beta$, we'll use the **test statistic**

\begin{align}
    \lambda(D; \theta) & = \frac{1}{50} \sqrt{\frac{1}{N} \sum_{n=1}^N \frac{[x_n - I_n(\theta)]^2}{I_n}} ,
\end{align}

where $D = x_1,\cdots,x_N$ are the observed infection counts and $I_n = I(t_n, \theta)$ is the predicted mean infection count at time $t_n$ found by solving the SIR equations

\begin{align}
    \frac{dS}{dt} & = - \beta S I ,\\
    \frac{dI}{dt} & = - \alpha I + \beta S I .
\end{align}

The test statistic $\lambda$ (which is large for $\theta$ values that are *disfavored* by the data) will be used to test hypotheses about the true value of $\theta$. 

The ODE model gives an approximate description of the evolution of the *mean* counts. (A more accurate model replaces the product $S I$ by the average $\langle S I \rangle$.) However, the fact that the ODE model is approximate is, in principle, not a problem. What matters in simulation-based inference is that the simulator $F_\theta$ is an accurate model of the data generation mechanism so that we can construct an accurate approximation of the distribution of the test statistic $\lambda$. 

### Critical region
In classical hypothesis testing the goal is to  *reject* hypotheses,  where $\alpha$, not to be confused with the parameter that appears in the SIR model,  is the probability to reject a true hypothesis. Rejecting a true hypothesis is obviously  a mistake; it is called a __Type 1 error__. The hypothesis $H_0: \theta = \theta_0$ is __rejected__ if the test statistic falls in the __critical region__ defined (in our case) by  $\lambda > C_\theta$, where $C_\theta$ is called the __critical value__.  If $\lambda_0$ falls in the complementary region $\lambda \le C_\theta$ then one has __failed to reject the hypothesis__. Then, one may choose to act as if the hypothesis $H_0: \theta = \theta_0$ has been accepted.

Clearly, we want to limit the chance of rejecting hypotheses that happen to be true, that is, we want to keep small the probability of a Type 1 error. The smaller the value of $\alpha$ the more stringent the condition for rejecting the hypothesis $H_0: \theta = \theta_0$. 

In medical research, the hypothesis $H_0: \theta = \theta_0$ is usually the so-called __null__ or __no effect__ hypothesis, for example, a treatment has no effect. The Type 1 error rate in that field is usually chosen to be $\alpha = 0.05$ (for purely historical reasons). Unfortunately, this is such a weak condition that many *no effect* hypotheses are falsely rejected leading many medical researchers to claim effects that invariably turn out to be marginal at best. Particle physicists are much more cautious about rejecting null hypotheses, that is, hypotheses stating that there is nothing new in the data. The null hypotheses are rejected with $\alpha = 2.7 \times 10^{-7}$. But even with such a stringent criterion, particle physicists are not immune from making the occasional false discovery claim.


### Confidence set
The set of $\theta$ values associated with hypotheses that failed to be rejected is called a __confidence set__ with __confidence level__ $\tau = 1 - \alpha$. The confidence set depends on the observed values $\lambda_0$ through the condition $\lambda_0 \le C_\theta$. Since (for a given $\theta$) $\lambda_0$ is an instance of a random variable, a given confidence set is likewise an instance of a __random set__ in the sense that the confidence set can change with each repetition of the experiment, or, in our case, epidemic. We say that confidence sets have __exact coverage__ if over an ensemble of experiments, or epidemics in our case, the fraction of confidence sets that contain the true value of $\theta$ *never* falls below the desired confidence level $1 - \alpha$. Note that the true value of $\theta$ does not have to be the same in each epidemic. 

Confidence sets that fall below the confidence level are said to __undercover__. Generally, we're happy if we have approximate coverage, that is, coverage that does not fall too far below the desired confidence level.
However, if the confidence sets have coverage that is well below the confidence level this is considered unsatisfactory because the true confidence level is then much lower than the claimed level of $1 - \alpha$.
 
### Simulated epidemics

The goal of this notebook is to simulate epidemics like the one that inflicted the English Boarding School, compute test statistics, and discrete random variables of the form 

$$Z = \mathbb{I}[\lambda(X^\prime; \theta ) \le \lambda_0(X; \theta)],$$

where $\mathbb{I}(x) = 1 \text{ if } x \text{ is true and } 0 \text{ otherwise}$, $X^\prime$ are simulated data, and  $\lambda_0$ is the "observed" value of $\lambda$ associated with "observed" data $X$. In Ref.[1], the observed data are the actual observations $D$; in the ALFFI approach, we simulate the "observed" data.

### References
  1. Ann Lee *et al.*, https://arxiv.org/abs/2107.03920

In [1]:
import os, sys
import numpy as np
import pandas as pd

# the standard modules for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

# to reload modules
import importlib

from tqdm import tqdm

# update fonts
FONTSIZE = 18
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# set usetex = False if LaTex is not 
# available on your system or if the 
# rendering is too slow
mp.rc('text', usetex=True)

# set a seed to ensure reproducibility
seed = 128
rnd  = np.random.RandomState(seed)

### Load SIR data and generate function

In [2]:
from SIR_genutil import generate, Fsolve, SIRdata
print(SIRdata)

 D           : [  3  25  75 227 296 258 236 192 126  71  28  11   7]
 I0          : 3
 O           : [  3  25  75 227 296 258 236 192 126  71  28  11   7]
 R0          : 0
 S0          : 763
 T           : [ 0  2  3  4  5  6  7  8  9 10 11 12 13]
 alpha0      : 0.465
 alpha_bins  : 16
 alpha_max   : 1.0
 alpha_min   : 0.0
 alpha_scale : 1.0
 beta0       : 0.00237
 beta_bins   : 16
 beta_max    : 0.7
 beta_min    : 0.2
 beta_scale  : 0.005
 model       : SIR
 scale       : 50
 tmax        : 14.0
 tmin        : 0.0



### Load $(\alpha, \beta)$ data

In [3]:
df = pd.read_csv('../data/SIR_alpha_beta_110k.csv.gz')
# N: number of epidemics to generate
N  = len(df)
print('number of entries: %d' % N)
df[:5]

number of entries: 110000


,alpha,beta
0,0.556824,0.432547
1,0.917183,0.617733
2,0.222595,0.684092
3,0.513685,0.231400
4,0.533168,0.343659


In [4]:
# df = pd.read_csv('../data/SIR_traindata_800k.csv')[:N].drop(labels='Unnamed: 0', axis=1)
# print(len(df), df.columns)
# df.to_csv('../data/SIR_traindata_100k.csv.gz', index=False)
# df[:5]

### Generate synthetic epidemics
We'll choose a uniform prior $\pi_\theta$ as our __proposal distribution__ over the parameter space.

In [ ]:
# length of data set
nD = len(SIRdata.D)

print(f'generate {N:d} epidemics')

# get randomly sampled parameters
alpha   = df.alpha.to_numpy()
beta    = df.beta.to_numpy()
I_means = [0]*N
I_counts= [0]*N

for j in tqdm(range(N)):
    
    # compute expected (i.e., mean) number of infections for current 
    # parameter point by solving coupled ODEs.
    # the parameters will be scaled to the correct values within Fsolve
    soln = Fsolve(alpha[j], beta[j])
    I_means[j] = list(soln.y[1])

    # generate data for one epidemic
    params  = (alpha[j], beta[j])
    _, i, _ = generate(params, SIRdata)
    
    # make list of simulated counts the same length as D
    I_counts[j] = list(i) + (nD-len(i))*[0] 

generate 110000 epidemics


 36%|████████████                      | 39052/110000 [2:20:03<14:15, 82.98it/s]

### Compute statistics
  1.  Shuffle epidemics to simulate "observed" data associated with randomly selected parameter points.
  1. For each parameter point $(\alpha, \beta)$, compute statistic $\lambda_o$ for randomly selected "observed" counts. 

In [11]:
def test_statistic(i, I):
    return np.sqrt(np.array([(d-f)**2/f for d, f in zip(i, I)]).mean()) / SIRdata.scale

In [12]:
li = np.zeros(N)
lo = np.zeros(N)
l0 = np.zeros(N)
Zo = np.zeros(N)
Z0 = np.zeros(N)

ii = np.arange(0, N, 1)
np.random.shuffle(ii)

for j in tqdm(range(N)):

    # compute test statistic for simulated data
    li[j] = test_statistic(I_counts[j], I_means[j])
    
    # compute test statistic for simulated "observed" data
    lo[j] = test_statistic(I_counts[ii[j]], I_means[j])
    Zo[j] = (li[j] <= lo[j]).astype(int)
    
    # compute test statistic for observed data
    l0[j] = test_statistic(SIRdata.D, I_means[j])
    Z0[j] = (li[j] <= l0[j]).astype(int)

100%|████████████████████████████████| 110000/110000 [00:02<00:00, 50314.97it/s]


### Reduce the number of significant places to shorten the output

In [13]:
Imeans = [[float(int(100*x))/100 for x in y] for y in I_means]

Convert lists to string representation.

In [14]:
i = [str(x) for x in I_counts]
I = [str(x) for x in Imeans]

### Write to CSV file

In [15]:
df = pd.DataFrame({'alpha': alpha, 
                   'beta': beta, 
                   'li': li, 
                   'lo': lo, 
                   'l0': l0, 
                   'Zo': Zo, 
                   'Z0': Z0, 
                   'i': i, 
                   'I': I})
df.to_csv('../data/SIR_traindata_110k.csv.gz', index=False, compression='gzip')
df[:5]

,alpha,beta,li,lo,l0,Zo,Z0,i,I
0,0.556824,0.432547,0.015810,0.629072,0.068463,1.0,1.0,"[3, 33, 61, 149, 208, 217, 181, 125, 85, 61, 3...","[3.0, 25.41, 66.24, 140.76, 212.17, 226.2, 190..."
1,0.917183,0.617733,0.037291,0.150635,0.323215,1.0,1.0,"[3, 19, 101, 180, 180, 125, 72, 38, 21, 7, 9, ...","[3.0, 46.43, 126.68, 188.65, 160.33, 101.22, 5..."
2,0.222595,0.684092,0.022214,0.545157,0.178384,1.0,1.0,"[3, 271, 514, 499, 413, 325, 265, 201, 155, 12...","[3.0, 232.07, 521.39, 511.27, 423.76, 341.88, ..."
3,0.513685,0.231400,0.098132,0.418606,0.630670,1.0,1.0,"[3, 6, 11, 23, 39, 63, 81, 97, 95, 98, 75, 59,...","[3.0, 6.22, 8.89, 12.59, 17.59, 24.14, 32.47, ..."
4,0.533168,0.343659,0.047897,0.071392,0.198700,1.0,1.0,"[3, 8, 19, 30, 52, 97, 150, 151, 124, 125, 106...","[3.0, 13.87, 28.69, 55.72, 97.06, 143.04, 172...."
